In [205]:
from __future__ import unicode_literals
from hazm import *
import pandas as pd
from collections import defaultdict
import numpy as np

# پروژه سوم
## هدف پروژه:
#### در این پروژه باید کامنت‌هایی که نشان دهنده پیشنهاد شدن محصول است را به وسیله داده‌هایی که از قبل داریم تخمین بزنیم.

## سوال ۱:
1. روش stemming: این روش نشانه‌های جمع و ضمیر افعال را حذف می‌کند.
2. روش lemmatization: این روش ریشه افعال را به جای افعال قرار می‌دهد. اشکالی که این روش دارد این است که تفاوتی بین فعل منفی و مثبت قائل نمی‌شود.
#### با توجه به نتیجه بدست آمده, روش lemmatization نتیجه بهتری داشته است. به دلیل اینکه زمان افعال برای آن مهم نیست و باعث می‌شود تعداد افعال یکسان بیشتر شود.

## سوال ۲:
### prior: احتمال recommended بودن یا نبودن یک نظر را نشان می‌دهد و با تقسیم تعداد نظرهای recommended به کل نظرها بدست می‌آید.
### posterior: احتمال recommended بودن نظر به شرط کلمات آمده را نشان می‌دهد و با فرمول بیز می‌توان با ضرب likelihood ها در prior بدست آورد.
### likelihood: احتمال اینکه کلمه‌ای در جایگاه i قرار بگیرد به شرط recommended بودن آن نظر را نشان می‌دهد. احتمال تمام جایگاه‌ها را یکسان درنظر می‌گیریم. بنابراین هر کلمه به شرط اینکه نظر recommended باشد احتمالی دارد که با تقسیم تعداد تکرار آن کلمه در این نوع نظرها تقسیم بر کل کلمات این نوع نظرات محاسبه می‌شود.
### evidence: این احتمال نشان دهنده احتمال تشکیل جمله‌های مختلف است. نیازی به محاسبهٔ این احتمال نیست زیرا برای همه ثابت است و می‌توان آنرا درنظر نگرفت.

## سوال ۳:
#### اگر کلمه ای در train وجود نداشته باشد, احتمال آن صفر درنظر گرفته می‌شود و با توجه به فرموال بیز, احتمال نهایی صفر می‌شود که دچار خطای زیادی می‌شود زیرا هر دو احتمال صفر می‌شوند یا اگر در یکی از دو نوع نظرات فقط وجود داشته باشد برای دیگری احتمال صفر دارد و قطعا آن یکی نظر را تخمین می‌زند.

## سوال ۴:
#### برای هر نظر احتمال likelihood را به این صورت تغییر می‌دهد: صورت همه احتمالات بدست آمده را به علاوه یک می‌کند و مخرج را با تعداد کلمه‌ها (جایگاه‌ها) جمع می‌کند و سپس تقسیم را انجام می‌دهد. در این صورت اگر کلمه‌ای از قبل در train وجود نداشته باشد, احتمال آن صفر نمی‌شود. این روش باعث بالا رفتن ارزیابی می‌شود.

In [206]:
TRAIN_PATH = "inputs/comment_train.csv"
TEST_PATH = "inputs/comment_test.csv"
TITLE = "title"
COMMENT = "comment"
RECOMMEND = "recommend"
RECOMMENDED = "recommended"
NOTRECOMMENDED = "not_recommended"
punctuations = ['٫', '.', '؟', '!', ';', '-', '_', '(', ')', '{', '}', '[', ']']

In [207]:
class Problem():
    def __init__(self, trainName, testName):
        self.train = pd.read_csv(trainName)
        self.test = pd.read_csv(testName)

        
        self.trainRowsNumber = self.train.shape[0]
        self.testRowsNumber = self.test.shape[0]
        
        self.trainRowTitleWords = []
        self.trainRowCommentWords = []
        
        self.testRowTitleWords = []
        self.testRowCommentWords = []
        
        self.recommendedTitleWords = defaultdict(int)
        self.recommendedCommentWords = defaultdict(int)
        
        self.notrecommendedTitleWords = defaultdict(int)
        self.notrecommendedCommentWords = defaultdict(int)
        
        self.recommended = np.log((self.train[RECOMMEND] == RECOMMENDED).sum() / self.trainRowsNumber)
        self.notrecommended = np.log((self.train[RECOMMEND] == NOTRECOMMENDED).sum() / self.trainRowsNumber)
        
        self.recommendCol = []
        self.wrongDetectedComments = []
        
        self.accuracy = 0
        self.precision = 0
        self.recall = 0
        self.f1 = 0
    
    def preProcess(self, _type):        
        if _type == "stemming":
            self.stemming()
        elif _type == "lemmatization":
            self.lemmatization()
    
    def normalize(self):
        normalizer = Normalizer()
        trainTitle = list(self.train[TITLE])
        trainComment = list(self.train[COMMENT])
                
        for i in range(self.trainRowsNumber):
            trainTitle[i] = normalizer.normalize(trainTitle[i])
            trainComment[i] = normalizer.normalize(trainComment[i])
            
#             for token in stopwords_list():
#                 trainTitle[i] = trainTitle[i].replace(token, " ")
#                 trainComment[i] = trainComment[i].replace(token, " ")
                
            for punc in punctuations:
                trainTitle[i] = trainTitle[i].replace(punc, "")
                trainComment[i] = trainComment[i].replace(punc, "")
                
        self.train[TITLE] = trainTitle
        self.train[COMMENT] = trainComment

        testTitle = list(self.test[TITLE])
        testComment = list(self.test[COMMENT])

        for i in range(self.testRowsNumber):
            testTitle[i] = normalizer.normalize(testTitle[i])
            testComment[i] = normalizer.normalize(testComment[i])

#             for token in stopwords_list():
#                 testTitle[i] = testTitle[i].replace(token, " ")
#                 testComment[i] = testComment[i].replace(token, " ")

            for punc in punctuations:
                testTitle[i] = testTitle[i].replace(punc, "")
                testComment[i] = testComment[i].replace(punc, "")
            
        self.test[TITLE] = testTitle
        self.test[COMMENT] = testComment
        
        
    def wordTokenize(self):
        trainTitle = list(self.train[TITLE])
        trainComment = list(self.train[COMMENT])
        for i in range(self.trainRowsNumber):
            self.trainRowTitleWords.append(word_tokenize(trainTitle[i]))
            self.trainRowCommentWords.append(word_tokenize(trainComment[i]))
            
        testTitle = list(self.test[TITLE])
        testComment = list(self.test[COMMENT])
        for i in range(self.testRowsNumber):
            self.testRowTitleWords.append(word_tokenize(testTitle[i]))
            self.testRowCommentWords.append(word_tokenize(testComment[i]))
            

            
    def stemming(self):
        stemmer = Stemmer()
        
        for i in range(self.trainRowsNumber):
            size = len(self.trainRowTitleWords[i])
            for j in range(size):
                self.trainRowTitleWords[i][j] = stemmer.stem(self.trainRowTitleWords[i][j])
                
            size = len(self.trainRowCommentWords[i])
            for j in range(size):
                self.trainRowCommentWords[i][j] = stemmer.stem(self.trainRowCommentWords[i][j])
        
        for i in range(self.testRowsNumber):
            size = len(self.testRowTitleWords[i])
            for j in range(size):
                self.testRowTitleWords[i][j] = stemmer.stem(self.testRowTitleWords[i][j])
                
            size = len(self.testRowCommentWords[i])
            for j in range(size):
                self.testRowCommentWords[i][j] = stemmer.stem(self.testRowCommentWords[i][j])
                                
    def lemmatization(self):
        lemmatizer = Lemmatizer()
        
        for i in range(self.trainRowsNumber):
            size = len(self.trainRowTitleWords[i])
            for j in range(size):
                self.trainRowTitleWords[i][j] = lemmatizer.lemmatize(self.trainRowTitleWords[i][j])
                
            size = len(self.trainRowCommentWords[i])
            for j in range(size):
                self.trainRowCommentWords[i][j] = lemmatizer.lemmatize(self.trainRowCommentWords[i][j])
        
        for i in range(self.testRowsNumber):
            size = len(self.testRowTitleWords[i])
            for j in range(size):
                self.testRowTitleWords[i][j] = lemmatizer.lemmatize(self.testRowTitleWords[i][j])
                
            size = len(self.testRowCommentWords[i])
            for j in range(size):
                self.testRowCommentWords[i][j] = lemmatizer.lemmatize(self.testRowCommentWords[i][j])

                
    def training(self):
        recommend = list(self.train[RECOMMEND])
        for i in range(self.trainRowsNumber):
            for j in range(len(self.trainRowTitleWords[i])):
                word = self.trainRowTitleWords[i][j]
                if recommend[i] == RECOMMENDED:
                    self.recommendedTitleWords[word] += 1
                else:
                    self.notrecommendedTitleWords[word] += 1

                    
            for j in range(len(self.trainRowCommentWords[i])):
                word = self.trainRowCommentWords[i][j]
                if recommend[i] == RECOMMENDED:
                    self.recommendedCommentWords[word] += 1
                else:
                    self.notrecommendedCommentWords[word] += 1
                    

    def process(self):
        totalRecommendedTitleWords = sum(self.recommendedTitleWords.values(), 0.0)
        totalRecommendedCommentWords = sum(self.recommendedCommentWords.values(), 0.0)
        totalNotRecommededTitleWords = sum(self.notrecommendedTitleWords.values(), 0.0)
        totalNotRecommendedCommentWords = sum(self.notrecommendedCommentWords.values(), 0.0)
        
        for i in range(self.testRowsNumber):
            pRecommended = self.recommended
            pNotRecommended = self.notrecommended
            for j in range(len(self.testRowTitleWords[i])):
                word = self.testRowTitleWords[i][j]
                pRecommended += np.log(self.recommendedTitleWords[word] / totalRecommendedTitleWords)
                pNotRecommended += np.log(self.notrecommendedTitleWords[word] / totalNotRecommededTitleWords)
            
            for j in range(len(self.testRowCommentWords[i])):
                word = self.testRowCommentWords[i][j]
                pRecommended += np.log(self.recommendedCommentWords[word] / totalRecommendedCommentWords)
                pNotRecommended += np.log(self.notrecommendedCommentWords[word] / totalNotRecommendedCommentWords)
                
            if pRecommended > pNotRecommended:
                self.recommendCol.append(RECOMMENDED)
            else:
                self.recommendCol.append(NOTRECOMMENDED)
            
    def processAdditive(self):
        totalRecommendedTitleWords = sum(self.recommendedTitleWords.values(), 0.0)
        totalRecommendedCommentWords = sum(self.recommendedCommentWords.values(), 0.0)
        totalNotRecommededTitleWords = sum(self.notrecommendedTitleWords.values(), 0.0)
        totalNotRecommendedCommentWords = sum(self.notrecommendedCommentWords.values(), 0.0)
        
        for i in range(self.testRowsNumber):
            pRecommended = self.recommended
            pNotRecommended = self.notrecommended
            size = len(self.testRowTitleWords[i])
            for j in range(size):
                word = self.testRowTitleWords[i][j]    

                pRecommended += np.log((self.recommendedTitleWords[word] + 1) / (totalRecommendedTitleWords + size))
                pNotRecommended += np.log((self.notrecommendedTitleWords[word] + 1) / (totalNotRecommededTitleWords + size))
            
            size = len(self.testRowCommentWords[i])
            for j in range(size):
                word = self.testRowCommentWords[i][j]
                pRecommended += np.log((self.recommendedCommentWords[word] + 1) / (totalRecommendedCommentWords + size))
                pNotRecommended += np.log((self.notrecommendedCommentWords[word] + 1) / (totalNotRecommendedCommentWords + size))
                
            if pRecommended > pNotRecommended:
                self.recommendCol.append(RECOMMENDED)
            else:
                self.recommendCol.append(NOTRECOMMENDED)
                
    def evaluate(self):
        recommend = list(self.test[RECOMMEND])
        title = list(self.test[TITLE])
        comment = list(self.test[COMMENT])
        correct = 0
        correctRecommended = 0
        totalRecommended = 0
        detectedRecommended = 0
        for i in range(self.testRowsNumber):
            if self.recommendCol[i] == recommend[i]:
                correct += 1
                if recommend[i] == RECOMMENDED:
                    correctRecommended += 1
            else:
                self.wrongDetectedComments.append((title[i], comment[i], recommend[i]))
                    
            if recommend[i] == RECOMMENDED:
                totalRecommended += 1
            
            if self.recommendCol[i] == RECOMMENDED:
                detectedRecommended += 1

        self.accuracy = correct / self.testRowsNumber * 100.0
        self.precision = correctRecommended / detectedRecommended * 100.0
        self.recall = correctRecommended / totalRecommended * 100.0
        self.f1 = 2 * (self.precision * self.recall) / (self.precision + self.recall)
                

## سوال ۵:
### مقدار precision دقت تشخیص نظرات recommended را نشان می‌شود. مثال زیر را در نظر بگیرید:
#### اگر تعداد کل نظرات recommended صدتا باشد و ما ۴ نظر را recommended تشخیص بدهیم که همه آنها هم درست باشند, دقت ما ۱۰۰ درصد است ولی تنها ۴ درصد نظرات را درست تشخیص داده ایم.
### مقدار recall  نسبت نظرات recommeded ای که ما تشخیص داده‌ایم به کل نظرات recommended موجود را نشان می‌دهد. به مثال زیر توجه کنید:
#### اگر ما ۱۰۰ نظر recommended و ۱۰۰ نظر not recommended داشته باشیم ولی تمام نظرات را recommended تشخیص دهیم, مقدار recall برابر ۱۰۰ درصد خواهد بود ولی دقت ما ۵۰ درصد می‌شود.
### بنابراین این دو مقدار با هم معنا پیدا می‌کنند.

## سوال ۶:
### میانگین گیری معکوس: اگر یکی از متغیرهای precision و recall مقدار کمی داشته باشد و دیگری مقدار زیادی داشته باشد, میانگین عادی این دو عدد نسبتا بزرگی می‌شود که اشتباه است. یا میانگین گیری معکوس این دو متغیر, میانگین به عددی که کوچکتر است نزدیکتر می‌شود و در نتیجه ارزیابی بهتری را نشان می‌دهد

## سوال ۷:

In [208]:
#A
problem = Problem(TRAIN_PATH, TEST_PATH)
problem.normalize()
problem.wordTokenize()
# problem.preProcess("lemmatization")
problem.training()
problem.processAdditive()
problem.evaluate()

wrongDetected = problem.wrongDetectedComments[:5]

print("Accuracy: ", problem.accuracy)
print("Precision: ", problem.precision)
print("Recall: ", problem.recall)
print("F1:", problem.f1)

Accuracy:  94.0
Precision:  93.56435643564357
Recall:  94.5
F1: 94.02985074626868


In [209]:
#A Lemmatization
problem = Problem(TRAIN_PATH, TEST_PATH)
problem.normalize()
problem.wordTokenize()
problem.preProcess("lemmatization")
problem.training()
problem.processAdditive()
problem.evaluate()

print("Accuracy: ", problem.accuracy)
print("Precision: ", problem.precision)
print("Recall: ", problem.recall)
print("F1:", problem.f1)

Accuracy:  93.5
Precision:  93.5
Recall:  93.5
F1: 93.5


In [210]:
#A Stemming
problem = Problem(TRAIN_PATH, TEST_PATH)
problem.normalize()
problem.wordTokenize()
problem.preProcess("stemming")
problem.training()
problem.processAdditive()
problem.evaluate()

print("Accuracy: ", problem.accuracy)
print("Precision: ", problem.precision)
print("Recall: ", problem.recall)
print("F1:", problem.f1)

Accuracy:  93.375
Precision:  93.26683291770573
Recall:  93.5
F1: 93.3832709113608


In [211]:
#B
problem = Problem(TRAIN_PATH, TEST_PATH)
# problem.normalize()
problem.wordTokenize()
# problem.preProcess("lemmatization")
problem.training()
problem.processAdditive()
problem.evaluate()

print("Accuracy: ", problem.accuracy)
print("Precision: ", problem.precision)
print("Recall: ", problem.recall)
print("F1:", problem.f1)

Accuracy:  94.125
Precision:  93.58024691358024
Recall:  94.75
F1: 94.1614906832298


In [212]:
#C
problem = Problem(TRAIN_PATH, TEST_PATH)
problem.normalize()
problem.wordTokenize()
problem.preProcess("lemmatization")
problem.training()
problem.process()
problem.evaluate()

print("Accuracy: ", problem.accuracy)
print("Precision: ", problem.precision)
print("Recall: ", problem.recall)
print("F1:", problem.f1)

Accuracy:  87.375
Precision:  95.16616314199395
Recall:  78.75
F1: 86.18331053351574


<ipython-input-207-5d6720a8fa2d>:167: RuntimeWarning: divide by zero encountered in log
  pNotRecommended += np.log(self.notrecommendedTitleWords[word] / totalNotRecommededTitleWords)
<ipython-input-207-5d6720a8fa2d>:166: RuntimeWarning: divide by zero encountered in log
  pRecommended += np.log(self.recommendedTitleWords[word] / totalRecommendedTitleWords)
<ipython-input-207-5d6720a8fa2d>:171: RuntimeWarning: divide by zero encountered in log
  pRecommended += np.log(self.recommendedCommentWords[word] / totalRecommendedCommentWords)
<ipython-input-207-5d6720a8fa2d>:172: RuntimeWarning: divide by zero encountered in log
  pNotRecommended += np.log(self.notrecommendedCommentWords[word] / totalNotRecommendedCommentWords)


In [213]:
#D
problem = Problem(TRAIN_PATH, TEST_PATH)
# problem.normalize()
problem.wordTokenize()
# problem.preProcess("lemmatization")
problem.training()
problem.process()
problem.evaluate()

print("Accuracy: ", problem.accuracy)
print("Precision: ", problem.precision)
print("Recall: ", problem.recall)
print("F1:", problem.f1)

Accuracy:  87.125
Precision:  96.26168224299066
Recall:  77.25
F1: 85.71428571428571


<ipython-input-207-5d6720a8fa2d>:167: RuntimeWarning: divide by zero encountered in log
  pNotRecommended += np.log(self.notrecommendedTitleWords[word] / totalNotRecommededTitleWords)
<ipython-input-207-5d6720a8fa2d>:166: RuntimeWarning: divide by zero encountered in log
  pRecommended += np.log(self.recommendedTitleWords[word] / totalRecommendedTitleWords)
<ipython-input-207-5d6720a8fa2d>:171: RuntimeWarning: divide by zero encountered in log
  pRecommended += np.log(self.recommendedCommentWords[word] / totalRecommendedCommentWords)
<ipython-input-207-5d6720a8fa2d>:172: RuntimeWarning: divide by zero encountered in log
  pNotRecommended += np.log(self.notrecommendedCommentWords[word] / totalNotRecommendedCommentWords)


## سوال ۸:
### همانطور که پیشبینی میشد با روش additive smoothing به نتیجه بهتری رسیدیم. ولی پیش پردازش باعث کم شدن دقت و F1 می‌شود. دلیل آن این است که همانطور که در سوال اول گفته شد, روش lemmatization کنار مزایایی که دارد اشکالاتی نیز دارد. 

In [214]:
for i in range(5):
    print(wrongDetected[i], end='\n\n')

('نقد پس از خرید', 'سلام، راحت شدم از کابل شارژ، توصیه میشود به شدت ارزان گوشی خود را به شارژ وایرلس مجهز کنید', 'recommended')

('خیالم راحت شد', 'فندک قبلیم مدام فیوز میسوزوند و یک بار شارژر موبایل هم سوزوند ولی با این هیچ مشکلی بوجود نیومده تا الان کیفیتش خیلی خوبه و لامپ هم داره', 'recommended')

('جنس و زیبایی', 'زیبا هستش از مدل\u200cهای دیگه مثل پارس … بنظرم زیبا\u200cتر با کیفیت\u200cتر هستش', 'recommended')

('بررسی فیلتر سرکان', 'من خودم جزو افرادی بودم که نزدیک سیزده ساله از انواع فیلتر سرکان اعم از روغن، هوا و اتاق استفاده میکردم ولی به تازگی متوجه و اطلاع یافتم که فیلتر گاج باکیفیت\u200cتر از فیلتر سرکان می\u200cباشد و هم چنین قیمت بمراتب مناسب تربت نسبت به سرکان دارد و طرف فروشنده که داشت روغن فیلترش را به من می\u200cفروخت واقعا بهم اثبات کرد که گاج باکیفیت\u200cتر از سرکان می\u200cباشد', 'not_recommended')

('کل صفحه رو نمیپوشونه', 'جنسش و چسبندگیش عالیه اما کل صفحه رو نمیپوشونه متاسفانه با این دید اگر بخرین، ناراضی نمیشید از خریدتون مثل من', 'not_recommended')



## سوال ۹:
### تعریف کردن از محصولات دیگر را نمی‌تواند تشخیص بدهد و به اشتباه فکر می‌کند تعاریف برای این محصول است و همینطور برعکس. اشکالات بقیه محصولات را گفتن باعث اشتباه در تشخیص می‌شود.
### اگر بتوان ریشه افعال را بدون از دست دادن منفی یا مثبت بودن آن نگه داشت می‌توان نتیجه را بهبود داد.